In [1]:
%matplotlib qt
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import os
import math

In [2]:
def txt_to_df(data_files):
    dfs = {}
    for filename in data_files:
        freq = filename.split('_')[1].split('Hz')[0]
        # print(f"Reading {freq}, {filename}...")
        with open(filename, 'r') as file:
            content = file.read()
            content = content.replace('"', '')
        
        with open(filename, 'w') as file:
            file.write(content)

        df = pd.read_csv(filename, delimiter=',')
    
        # print("Number of NaNs in each column:")
        # print(df.isnull().sum())
        df = df.dropna()
        
        df['Error'] = df[' Error']
        df['Target angle'] = df[' Target angle']
        df.drop(columns=[' Error', ' Target angle'], inplace=True)

        for col in df.columns:
            try:
                df[col] = pd.to_numeric(df[col], errors='raise')
            except ValueError as e:
                # print(f"\nOh Oh! Conversion failed in column: {col}, Error: {e}\n")
                df[col] = pd.to_numeric(df[col], errors='coerce')

        # print("Number of NaNs in each column:")
        # print(df.isnull().sum())
        df = df.dropna()
        
        dfs[freq] = df

    return dfs

In [3]:

def sine_wave(t, A, phase, offset, freq):
    return A * np.sin(2 * np.pi * freq * t + phase) + offset


def get_rms_amps(data, freq, ax_target, ax_error):
    time_ms = data['Run time']
    time_ms = pd.to_numeric(time_ms, errors='raise')
    time_s = time_ms / 1000

    error = data['Error']
    target_angle = data['Target angle']
    
    if freq == 0.2:
        target_angle = target_angle[1:]

    ### I'm so sorry for this code. I know this can be done better but hey, it works!
    if max(target_angle) >= 30:
        print(freq, "30")
        start_index = np.argmax(target_angle > 30)
    elif max(target_angle) >= 27:
        print(freq, '27')
        start_index = np.argmax(target_angle >= 27)
    elif max(target_angle) >= 25:
        print(freq, '25')
        start_index = np.argmax(target_angle >= 25)
    elif max(target_angle) >= 24:
        print(freq, '24', max(target_angle))
        start_index = np.argmax(target_angle >= 24)
    elif max(target_angle) >= 23:
        print(freq, '23', max(target_angle))
        start_index = np.argmax(target_angle >= 23)
        print('start', start_index, target_angle.iloc[start_index])


    angle_range = 60
    cycles = 10 if freq in [0.1, 0.01] else 5
    points_per_wave = 500
    total_points = cycles * points_per_wave
    t_perfect = np.linspace(0, cycles / freq, total_points)
    perfect_wave = (angle_range/2) * np.cos(2 * np.pi * freq * t_perfect)
    t_perfect_shifted = t_perfect + time_s[start_index]
    # plt.figure()

    if freq == 0.2:
        time_s = time_s[1:]

    params_target, _ = curve_fit(
        lambda t, A, phase, offset: sine_wave(t, A, phase, offset, freq), time_s, target_angle, p0=[30, 0, 0]
        )
    A_target, phase_target, offset_target = params_target
    target_fit = sine_wave(time_s, A_target, phase_target, offset_target, freq)

    params_perfect, _ = curve_fit(
        lambda t, A, phase, offset: sine_wave(t, A, phase, offset, freq), t_perfect_shifted, perfect_wave, p0=[30, 0, 0]
        )
    A_perfect, phase_perfect, offset_perfect = params_perfect
    perfect_fit = sine_wave(t_perfect_shifted, A_perfect, phase_perfect, offset_perfect, freq)
    
    phase_diff_rad = phase_target - phase_perfect
    phase_diff_deg = phase_diff_rad * (180 / np.pi)
    # print(len(perfect_fit), len(perfect_wave), len(t_perfect_shifted)) # 5129 5000 5000

    ax_target.plot(time_s, target_angle, label='Target Angle')
    # ax_target.plot(time_s, target_fit, label='Target Fit', linestyle='--')
    ax_target.plot(t_perfect_shifted, perfect_wave, label='Perfect Angle', alpha=0.8)
    # ax_target.set_xlabel('Time (s)', fontsize=6)
    # ax_target.set_ylabel('Angle (degrees)', fontsize=6)
    ax_target.set_title(f"Target Angle at {freq}Hz", fontsize=6)
    ax_target.tick_params(axis="both", labelsize=6)
    # ax_target.legend(fontsize=6, loc='lower right')

    # ax_error.plot(time_s, error, label='Error')
    # ax_error.plot(time_s, error_fit, label='Error Fit')
    # ax_error.set_xlabel('Time (s)', fontsize=6)
    # ax_error.set_ylabel('Angle (degrees)', fontsize=6)
    # ax_error.set_title(f"Error at {freq}Hz", fontsize=6)
    # ax_error.tick_params(axis="both", labelsize=6)
    # ax_error.legend(fontsize=6, loc='lower right')

    rms_target = np.mean(target_angle**2)**0.5
    rms_error = np.mean(error**2)**0.5
    # print(f"{freq}Hz: target amplitude - {rms_target}, error amplitude - {rms_error}")

    return rms_target, rms_error, phase_diff_deg





In [4]:
def plot_rms(rms_targets, rms_errors, freqs):
    print(freqs)
    print(type(freqs))
    print(type(rms_targets))
    print('done')
    combined = sorted(zip(freqs, rms_targets, rms_errors))
    freqs, rms_targets, rms_errors = zip(*combined)

    fig, ax = plt.subplots()
    ax.plot(freqs, rms_targets, label='Target RMS', linestyle='-', marker='x')
    ax.plot(freqs, rms_errors, label='Error RMS', linestyle='-', marker='x')
    ax.set_xlabel('Frequency (Hz)', fontsize=10)
    ax.set_ylabel('Amplitude (degrees)', fontsize=10)
    ax.set_title('RMS Amplitude vs Frequency', fontsize=10)
    ax.tick_params(axis="both", labelsize=10)
    ax.legend(fontsize=10, loc='upper right')
    plt.grid()
    plt.show()

In [5]:
def bode_plot(gains, phases, freqs):
    gains = gains[0]
    sorted_tuples = sorted(zip(freqs, gains, phases))
    freqs, gains, phases = zip(*sorted_tuples)
    print(f"Freqs: {freqs}")
    print(f"Gains: {gains}")
    print(f"Phases: {phases}")
    gain_db = 20 * np.log10(gains)
    plt.figure()

    # Magnitude plot
    plt.semilogx(freqs, gain_db, 'x-')
    # plt.plot(freqs, gains, 'x-')
    plt.title("Gain vs Frequency")
    plt.ylabel("Gain (dB)")
    plt.xlabel("Frequency (Hz)")
    plt.grid()

    plt.figure()
    plt.semilogx(freqs, phases, 'x-')
    # plt.plot(freqs, phases, 'x-')
    plt.title("Phase vs Frequency")
    plt.ylabel("Phase (degrees)")
    plt.xlabel("Frequency (Hz)")
    plt.grid()

    plt.tight_layout()
    plt.show()

In [6]:
def run(data_files):
    plt.close('all')
    data = txt_to_df(data_files)
    rms_targets = []
    rms_errors = []
    freqs = []
    n_freqs = len(data_files)
    phase_diffs = []
    # print(np.ceil(n_freqs/2))
    fig, axs = plt.subplots(int(np.ceil(n_freqs/2)), 4, figsize=(12, 3 * n_freqs))

    for i, (freq, df) in enumerate(data.items()):
        # if freq == "0.07":
        # print('\n', float(freq))

        row = i // 2
        col = i % 2
        if i % 2 == 1:
            col += 1
        rms_target, rms_error, phase_diff = get_rms_amps(df, float(freq), axs[row, col], axs[row, col+1])
        rms_targets.append(rms_target)
        rms_errors.append(rms_error)
        freqs.append(float(freq))
        phase_diffs.append(phase_diff)
            # break
    plt.tight_layout()
    plot_rms(rms_targets, rms_errors, freqs)
    gains = [np.array(rms_targets)/(30/2**0.5)]
    bode_plot(gains, phase_diffs, freqs)

data_files = os.listdir("data")
data_files = sorted([f"data/{f}" for f in data_files])
# print(sorted(data_files))
run(data_files)

0.01 30
0.02 30
0.03 30
0.05 27
0.06 27
0.07 27
0.08 27
0.09 27
0.11 25
0.12 25
0.13 25
0.14 25
0.15 25
0.16 24 24.43798
0.17 24 24.35589
0.18 23 23.60445
start 10 23.23773
0.19 23 23.95102
start 7 23.41088
0.1 30
0.2 23 23.90603
start 0 23.27929


/var/folders/t0/9s1kd78j2w76jjypvc2j3x900000gn/T/ipykernel_22554/604742813.py:53: OptimizeWarning: Covariance of the parameters could not be estimated
  params_perfect, _ = curve_fit(


[0.01, 0.02, 0.03, 0.05, 0.06, 0.07, 0.08, 0.09, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.1, 0.2]
<class 'list'>
<class 'list'>
done
Freqs: (0.01, 0.02, 0.03, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2)
Gains: (np.float64(0.9814450430337361), np.float64(0.9706501705308027), np.float64(1.0032199829453365), np.float64(0.9865910951025113), np.float64(0.9694899816655056), np.float64(0.9513481364919602), np.float64(0.9258502932117829), np.float64(0.8800487562697279), np.float64(0.8711187260228173), np.float64(0.7870877638179815), np.float64(0.7672856930663948), np.float64(0.7533791432891578), np.float64(0.7242434137175732), np.float64(0.7324188498899759), np.float64(0.6835316566535696), np.float64(0.7091689847413672), np.float64(0.6467405673560208), np.float64(0.6427368111222165), np.float64(0.6744365763397713))
Phases: (np.float64(-22.24759989099289), np.float64(-19.166137643996787), np.float64(-17.083759613685263), np.float6